In [18]:
import pandas as pd
pd.set_option("display.max_row", None)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
from absbox import Generic

bondGrpExample = Generic(
    "TEST01"
    ,{"cutoff":"2021-03-01","closing":"2021-06-15","firstPay":"2021-07-26"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthEnd","stated":"2030-01-01"}
    ,{'assets':[["Mortgage"
        ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":30
          ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
          ,{"currentBalance":2200
          ,"currentRate":0.08
          ,"remainTerm":20
          ,"status":"current"}]]}
    ,(("acc01",{"balance":0}),)
    ,(("A",{"A-1":
            {"balance":400
             ,"rate":0.09
             ,"originBalance":400
             ,"originRate":0.07
             ,"startDate":"2021-06-15"
             ,"rateType":{"floater":[0.05, "SOFR1Y",-0.0169,"YearEnd"]}
             ,"bondType":{"Sequential":None}
             ,"maturityDate":"2025-01-01"}
            ,"A-2":
            {"balance":600
             ,"rate":0.08
             ,"originBalance":600
             ,"originRate":0.07
             ,"startDate":"2021-06-15"
             ,"rateType":{"Fixed":0.08}
             ,"bondType":{"Sequential":None}
            ,"maturityDate":"2026-01-01"}           
           
           })
      ,("B",{"balance":1000
             ,"rate":0.0
             ,"originBalance":1000
             ,"originRate":0.07
             ,"startDate":"2020-01-03"
             ,"rateType":{"Fixed":0.00}
             ,"bondType":{"Equity":None}
             }))
    ,(("trusteeFee",{"type":{"fixFee":30}}),)
    ,{"amortizing":[
         ["payFee","acc01",['trusteeFee']]
         #,["calcIntByGroup",["A"]]
         ,["accrueAndPayIntByGroup","acc01","A", "byName"]
         ,["payPrinByGroup","acc01","A","byMaturity"]
         ,["payPrin","acc01",["B"]]
         ,["payIntResidual","acc01","B"]
     ]}
    ,[["CollectedInterest","acc01"]
      ,["CollectedPrincipal","acc01"]
      ,["CollectedPrepayment","acc01"]
      ,["CollectedRecoveries","acc01"]]
    ,None
    ,None
    ,None
    ,None
    ,("PreClosing","Amortizing")
    )

In [56]:
from absbox import API,EnginePath

localAPI = API(EnginePath.LOCAL,check=False )

Connecting engine server -> http://localhost:8081

✅Connected, local lib:0.28.7, server:0.28.12

In [85]:
r = localAPI.run(bondGrpExample
               ,poolAssump = None
               ,runAssump = [("interest",("SOFR1Y",[["2021-10-01",0.025]
                                                   ,["2022-02-01",0.029]]))
                            ,("inspect",("MonthEnd",("bondRate","A"))
                                       ,("MonthEnd",("bondWaRate","A-1","A-2")))
                            ,("report",{"dates":"MonthEnd"})]
               ,read = True)

Warning Message from server:

In [86]:
r['result']['inspect']

{'<BondRate>':             <BondRate>
 Date                  
 2021-03-01    0.084000
 2021-03-31    0.084000
 2021-04-30    0.084000
 2021-05-31    0.084000
 2021-06-30    0.084000
 2021-07-31    0.084000
 2021-08-31    0.084000
 2021-09-30    0.084000
 2021-10-31    0.084000
 2021-11-30    0.084000
 2021-12-31    0.051240
 2022-01-31    0.051240
 2022-02-28    0.052850
 2022-03-31    0.058849
 2022-04-30    0.066679
 2022-05-31    0.077390
 2022-06-30    0.080000
 2022-07-31    0.080000
 2022-08-31    0.080000
 2022-09-30    0.080000
 2022-10-31    0.080000
 2022-11-30    0.000000
 2022-12-31    0.000000
 2023-01-31    0.000000
 2023-02-28    0.000000
 2023-03-31    0.000000
 2023-04-30    0.000000
 2023-05-31    0.000000
 2023-06-30    0.000000
 2023-07-31    0.000000
 2023-08-31    0.000000,
 '<BondWaRate:A-1,A-2>':             <BondWaRate:A-1,A-2>
 Date                            
 2021-03-01              0.084000
 2021-03-31              0.084000
 2021-04-30              0.084000

In [50]:
from absbox import readBondsCf,readFeesCf,readAccsCf

In [61]:
0.4*0.09+0.6*0.08

0.08399999999999999

In [51]:
readBondsCf(r['bonds'])

BondGroup        A                                                      \
Bond           A-1                                        A-2            
Field      balance interest principal    rate    cash balance interest   
date                                                                     
2021-07-26  400.00     0.00      0.00  0.0900    0.00  600.00     0.00   
2021-08-20  400.00     0.00      0.00  0.0900    0.00  600.00     0.00   
2021-09-20  400.00     0.00      0.00  0.0900    0.00  600.00     0.00   
2021-10-20  400.00     0.00      0.00  0.0900    0.00  600.00     0.00   
2021-11-20  400.00     0.00      0.00  0.0900    0.00  600.00     0.00   
2021-12-20  400.00     0.00      0.00  0.0900    0.00  600.00     0.00   
2022-01-20  400.00     0.00      0.00  0.0081    0.00  600.00     0.00   
2022-02-20  364.01    19.04     35.99  0.0081   55.03  600.00    32.83   
2022-03-20  250.05     0.22    113.96  0.0081  114.18  600.00     3.68   
2022-04-20  136.43     0.17    113.62  0.0081  113.79  600.00     4.07   
2022-05-20   22.60     0.09    113.83  0.0081  113.92  600.00     3.94   
2022-06-20    0.00     0.01     22.60  0.0081   22.61  508.83     4.07   
2022-07-20     NaN      NaN       NaN     NaN     NaN  394.31     3.34   
2022-08-20     NaN      NaN       NaN     NaN     NaN  279.13     2.67   
2022-09-20     NaN      NaN       NaN     NaN     NaN  163.17     1.89   
2022-10-20     NaN      NaN       NaN     NaN     NaN   46.38     1.07   
2022-11-20     NaN      NaN       NaN     NaN     NaN    0.00     0.31   
2022-12-20     NaN      NaN       NaN     NaN     NaN     NaN      NaN   
2023-01-20     NaN      NaN       NaN     NaN     NaN     NaN      NaN   
2023-02-20     NaN      NaN       NaN     NaN     NaN     NaN      NaN   
2023-03-20     NaN      NaN       NaN     NaN     NaN     NaN      NaN   
2023-04-20     NaN      NaN       NaN     NaN     NaN     NaN      NaN   
2023-05-20     NaN      NaN       NaN     NaN     NaN     NaN      NaN   
2023-06-20     NaN      NaN       NaN     NaN     NaN     NaN      NaN   
2023-07-20     NaN      NaN       NaN     NaN     NaN     NaN      NaN   
2023-08-20     NaN      NaN       NaN     NaN     NaN     NaN      NaN   
2023-09-20     NaN      NaN       NaN     NaN     NaN     NaN      NaN   

BondGroup                                 B                                  
Bond                                      -                                  
Field      principal  rate    cash  balance interest principal rate    cash  
date                                                                         
2021-07-26      0.00  0.08    0.00  1000.00     0.00      0.00    0    0.00  
2021-08-20      0.00  0.08    0.00  1000.00     0.00      0.00    0    0.00  
2021-09-20      0.00  0.08    0.00  1000.00     0.00      0.00    0    0.00  
2021-10-20      0.00  0.08    0.00  1000.00     0.00      0.00    0    0.00  
2021-11-20      0.00  0.08    0.00  1000.00     0.00      0.00    0    0.00  
2021-12-20      0.00  0.08    0.00  1000.00     0.00      0.00    0    0.00  
2022-01-20      0.00  0.08    0.00  1000.00     0.00      0.00    0    0.00  
2022-02-20      0.00  0.08   32.83  1000.00     0.00      0.00    0    0.00  
2022-03-20      0.00  0.08    3.68  1000.00     0.00      0.00    0    0.00  
2022-04-20      0.00  0.08    4.07  1000.00     0.00      0.00    0    0.00  
2022-05-20      0.00  0.08    3.94  1000.00     0.00      0.00    0    0.00  
2022-06-20     91.17  0.08   95.24  1000.00     0.00      0.00    0    0.00  
2022-07-20    114.52  0.08  117.86  1000.00     0.00      0.00    0    0.00  
2022-08-20    115.18  0.08  117.85  1000.00     0.00      0.00    0    0.00  
2022-09-20    115.96  0.08  117.85  1000.00     0.00      0.00    0    0.00  
2022-10-20    116.79  0.08  117.86  1000.00     0.00      0.00    0    0.00  
2022-11-20     46.38  0.08   46.69   928.84     0.00     71.16    0   71.16  
2022-12-20       NaN   NaN     NaN   810.98     0.00    117.86    0

In [52]:
r['bonds']['A']['A-1']

,balance,interest,principal,rate,cash,intDue,intOverInt,factor,memo
date,,,,,,,,,
2021-07-26,400.00,0.00,0.00,0.0900,0.00,4.04,0,1.000000,"[<PayInt:A-1>, <PayPrin:A-1>]"
2021-08-20,400.00,0.00,0.00,0.0900,0.00,6.50,0,1.000000,"[<PayInt:A-1>, <PayPrin:A-1>]"
2021-09-20,400.00,0.00,0.00,0.0900,0.00,9.55,0,1.000000,"[<PayInt:A-1>, <PayPrin:A-1>]"
2021-10-20,400.00,0.00,0.00,0.0900,0.00,12.50,0,1.000000,"[<PayInt:A-1>, <PayPrin:A-1>]"
2021-11-20,400.00,0.00,0.00,0.0900,0.00,15.55,0,1.000000,"[<PayInt:A-1>, <PayPrin:A-1>]"
2021-12-20,400.00,0.00,0.00,0.0900,0.00,18.50,0,1.000000,"[<PayInt:A-1>, <PayPrin:A-1>]"
2022-01-20,400.00,0.00,0.00,0.0081,0.00,18.77,0,1.000000,"[<PayInt:A-1>, <PayPrin:A-1>]"
2022-02-20,364.01,19.04,35.99,0.0081,55.03,0.00,0,0.910025,"[<PayInt:A-1>, <PayPrin:A-1>]"
2022-03-20,250.05,0.22,113.96,0.0081,114.18,0.00,0,0.625125,"[<PayInt:A-1>, <PayPrin:A-1>]"


In [36]:
readBondsCf(r['bonds'])

BondGroup        A                                                      \
Bond           A-1                                        A-2            
Field      balance interest principal    rate    cash balance interest   
date                                                                     
2021-07-26  400.00     0.00      0.00  0.0900    0.00  600.00     0.00   
2021-08-20  400.00     0.00      0.00  0.0900    0.00  600.00     0.00   
2021-09-20  400.00     0.00      0.00  0.0900    0.00  600.00     0.00   
2021-10-20  400.00     0.00      0.00  0.0900    0.00  600.00     0.00   
2021-11-20  400.00     0.00      0.00  0.0900    0.00  600.00     0.00   
2021-12-20  400.00     0.00      0.00  0.0900    0.00  600.00     0.00   
2022-01-20  400.00     0.00      0.00  0.0081    0.00  600.00     0.00   
2022-02-20  364.01    19.04     35.99  0.0081   55.03  600.00    32.83   
2022-03-20  250.05     0.22    113.96  0.0081  114.18  600.00     3.68   
2022-04-20  136.43     0.17    113.62  0.0081  113.79  600.00     4.07   
2022-05-20   22.60     0.09    113.83  0.0081  113.92  600.00     3.94   
2022-06-20    0.00     0.01     22.60  0.0081   22.61  508.83     4.07   
2022-07-20     NaN      NaN       NaN     NaN     NaN  394.31     3.34   
2022-08-20     NaN      NaN       NaN     NaN     NaN  279.13     2.67   
2022-09-20     NaN      NaN       NaN     NaN     NaN  163.17     1.89   
2022-10-20     NaN      NaN       NaN     NaN     NaN   46.38     1.07   
2022-11-20     NaN      NaN       NaN     NaN     NaN    0.00     0.31   
2022-12-20     NaN      NaN       NaN     NaN     NaN     NaN      NaN   
2023-01-20     NaN      NaN       NaN     NaN     NaN     NaN      NaN   
2023-02-20     NaN      NaN       NaN     NaN     NaN     NaN      NaN   
2023-03-20     NaN      NaN       NaN     NaN     NaN     NaN      NaN   
2023-04-20     NaN      NaN       NaN     NaN     NaN     NaN      NaN   
2023-05-20     NaN      NaN       NaN     NaN     NaN     NaN      NaN   
2023-06-20     NaN      NaN       NaN     NaN     NaN     NaN      NaN   
2023-07-20     NaN      NaN       NaN     NaN     NaN     NaN      NaN   
2023-08-20     NaN      NaN       NaN     NaN     NaN     NaN      NaN   
2023-09-20     NaN      NaN       NaN     NaN     NaN     NaN      NaN   

BondGroup                                 B                                  
Bond                                      -                                  
Field      principal  rate    cash  balance interest principal rate    cash  
date                                                                         
2021-07-26      0.00  0.08    0.00  1000.00     0.00      0.00    0    0.00  
2021-08-20      0.00  0.08    0.00  1000.00     0.00      0.00    0    0.00  
2021-09-20      0.00  0.08    0.00  1000.00     0.00      0.00    0    0.00  
2021-10-20      0.00  0.08    0.00  1000.00     0.00      0.00    0    0.00  
2021-11-20      0.00  0.08    0.00  1000.00     0.00      0.00    0    0.00  
2021-12-20      0.00  0.08    0.00  1000.00     0.00      0.00    0    0.00  
2022-01-20      0.00  0.08    0.00  1000.00     0.00      0.00    0    0.00  
2022-02-20      0.00  0.08   32.83  1000.00     0.00      0.00    0    0.00  
2022-03-20      0.00  0.08    3.68  1000.00     0.00      0.00    0    0.00  
2022-04-20      0.00  0.08    4.07  1000.00     0.00      0.00    0    0.00  
2022-05-20      0.00  0.08    3.94  1000.00     0.00      0.00    0    0.00  
2022-06-20     91.17  0.08   95.24  1000.00     0.00      0.00    0    0.00  
2022-07-20    114.52  0.08  117.86  1000.00     0.00      0.00    0    0.00  
2022-08-20    115.18  0.08  117.85  1000.00     0.00      0.00    0    0.00  
2022-09-20    115.96  0.08  117.85  1000.00     0.00      0.00    0    0.00  
2022-10-20    116.79  0.08  117.86  1000.00     0.00      0.00    0    0.00  
2022-11-20     46.38  0.08   46.69   928.84     0.00     71.16    0   71.16  
2022-12-20       NaN   NaN     NaN   810.98     0.00    117.86    0

In [73]:
r['accounts']['acc01']

,balance,change,memo
date,,,
2021-06-15,0.00,0.00,<Pool:CollectedRecoveries>
2021-06-15,0.00,0.00,<Pool:CollectedPrepayment>
2021-06-15,0.00,0.00,<Pool:CollectedPrincipal>
2021-06-15,0.00,0.00,<Pool:CollectedInterest>
2021-06-30,0.00,0.00,<Pool:CollectedRecoveries>
2021-06-30,0.00,0.00,<Pool:CollectedPrepayment>
2021-06-30,0.00,0.00,<Pool:CollectedPrincipal>
2021-06-30,0.00,0.00,<Pool:CollectedInterest>
2021-07-26,0.00,0.00,<SeqPayFee:trusteeFee>


In [63]:
r['bonds'].keys()

dict_keys(['A', 'B'])

In [60]:
r['bonds'] & lens.Recur(pd.DataFrame).get()

,balance,interest,principal,rate,cash,factor,memo
date,,,,,,,
2021-07-26,400.00,0,0.00,0,0.00,1.00000,<PayPrin:A-1>
2021-08-20,400.00,0,0.00,0,0.00,1.00000,<PayPrin:A-1>
2021-09-20,400.00,0,0.00,0,0.00,1.00000,<PayPrin:A-1>
2021-10-20,400.00,0,0.00,0,0.00,1.00000,<PayPrin:A-1>
2021-11-20,400.00,0,0.00,0,0.00,1.00000,<PayPrin:A-1>
2021-12-20,400.00,0,0.00,0,0.00,1.00000,<PayPrin:A-1>
2022-01-20,400.00,0,0.00,0,0.00,1.00000,<PayPrin:A-1>
2022-02-20,364.86,0,35.14,0,35.14,0.91215,<PayPrin:A-1>
2022-03-20,317.72,0,47.14,0,47.14,0.79430,<PayPrin:A-1>


In [67]:
readBondsCf(r['bonds'])

TypeError: unhashable type: 'list'